# Cation exchange competition among Na<sup>+</sup>, K<sup>+</sup> and Ca<sup>2+</sup>

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on Mar 10th, 2022.<br>Last revised on Mar 16, 2023 by Allan Leal.</p>

```{attention}
Always make sure you are using the [latest version of Reaktoro](https://anaconda.org/conda-forge/reaktoro). Otherwise, some new features documented on this website will not work on your machine and you may receive unintuitive errors. Follow these [update instructions](updating_reaktoro_via_conda) to get the latest version of Reaktoro!
```

In this tutorial, we show how Reaktoro can be used to model ion exchange processes. We use the PHREEQC database `phreeqc.dat` for this purpose. We assume that there is an ion exchanger X<sup>-</sup> on which the aqueous cations Na<sup>+</sup>, K<sup>+</sup> and Ca<sup>2+</sup> can bind and form the exchange phases NaX, KX and CaX<sub>2</sub> respectively. We are interested in calculating the amounts of these phases as the concentrations of K<sup>+</sup> and Ca<sup>2+</sup> vary in the aqueous solution.

First, we define a chemical system representative of the process we want to model.

In [47]:
import numpy as np
from reaktoro import *
import pandas as pd

# Initialize the PHREEQC database
db = PhreeqcDatabase("phreeqc.dat")

# Define an aqueous phase
solution = AqueousPhase("H2O Na+ Cl- H+ OH- K+ Ca+2 Mg+2")
solution.set(ActivityModelPhreeqc(db))

# Define an ion exchange phase
exchange = IonExchangePhase("NaX KX CaX2")
exchange.set(ActivityModelIonExchangeGainesThomas())

# Create the chemical system
system = ChemicalSystem(db, solution, exchange)

As we are going to perform a sequence of chemical equilibrium calculations, we define an object of {{EquilibriumSolver}} as well as {{IonExchangeProps}} and {{AqueousProps}} using the chemical system. These objects will be updated after every equilibrium calculation.

In [48]:
# Define the equilibrium solver
solver = EquilibriumSolver(system)

# Define exchange, aqueous, and chemical properties
exprops = IonExchangeProps(system)
aqprops = AqueousProps(system)

```{note}
Note that at the end of each equilibrium calculation, `state.props()` can be used to access a {{ChemicalProps}} object with the chemical properties of the system evaluated in the last iteration of the algorithm.
```

Next, we define the range of Ca<sup>2+</sup> and K<sup>+</sup> initial amounts as well as auxiliary `pandas.DataFrame` to collect the results of the ion exchange simulations.

In [49]:
# Output dataframe
df = pd.DataFrame(columns=[
    "amount_K", 
    "amount_Ca",
    "amount_K_ion", 
    "amount_Na_ion", 
    "amount_Ca_ion",
    "amount_KX", 
    "amount_NaX", 
    "amount_CaX2",
    "pH"
])

# Sampling arrays storing the amounts of ions
steps = 21

mols_K  = np.linspace(0.1, 0.0, num=steps)
mols_Ca = np.linspace(0.0, 0.5, num=steps)

We'll now perform a sequence of equilibrium calculations by varying the initial amounts of ions Ca<sup>2+</sup> and K<sup>+</sup> ions (taken from the arrays `mols_Ca` and `mols_K`, respectively). At the end of each calculation, we'll extract the following properties from the computed chemical state:
* amount of K,
* amount of Ca,
* amount of K<sup>+</sup>,
* amount of Na<sup>+</sup>,
* amount of Ca<sup>2+</sup>,
* amount of KX,
* amount of NaX,
* amount of CaX2,
* pH.

In [50]:
for mol_K, mol_Ca in zip(mols_K, mols_Ca):

    # Define initial chemical state
    state = ChemicalState(system)
    state.setTemperature(25, "celsius")
    state.setPressure(1, "atm")
    state.set("H2O" , 1.0   , "kg")
    state.set("NaX" , 0.4   , "mol")
    state.set("K+"  , mol_K , "mol")
    state.set("Ca+2", mol_Ca, "mol")

    # Equilibrate the chemical state
    res = solver.solve(state)
    
    # Stop if the equilibration did not converge or failed
    assert res.succeeded()

    # Update exchange and aqueous properties
    chemprops = state.props()

    exprops.update(chemprops)
    aqprops.update(chemprops)

    # Update output arrays:
    df.loc[len(df)] = [
        chemprops.elementAmount('K'),   # amount_K
        chemprops.elementAmount('Ca'),  # amount_Ca
        state.speciesAmount('K+'),      # amount_K+
        state.speciesAmount('Na+'),     # amount_Na+
        state.speciesAmount('Ca+2'),    # amount_Ca+2
        state.speciesAmount('KX'),      # amount_KX
        state.speciesAmount('NaX'),     # amount_NaX
        state.speciesAmount('CaX2'),    # amount_CaX2
        aqprops.pH()                    # pH
    ]

The data collected in the previous calculations can be inspected by printing the `pandas.DataFrame` object `df`:

In [51]:
df

,amount_K,amount_Ca,amount_K_ion,amount_Na_ion,amount_Ca_ion,amount_KX,amount_NaX,amount_CaX2,pH
0,0.1,2e-16,0.00571081,0.0942892,1e-16,0.0942892,0.305711,1e-16,6.99674
1,0.095,0.025,0.00857519,0.134804,0.000810585,0.0864248,0.265196,0.0241894,6.99694
2,0.09,0.05,0.0114588,0.17168,0.0034304,0.0785412,0.22832,0.0465696,6.99732
3,0.085,0.075,0.014057,0.20273,0.00910655,0.070943,0.19727,0.0658934,6.99787
4,0.08,0.1,0.0160688,0.227017,0.0184573,0.0639312,0.172983,0.0815427,6.9986
5,0.075,0.125,0.0173796,0.245317,0.0311517,0.0576204,0.154683,0.0938483,6.99952
6,0.07,0.15,0.0180442,0.259116,0.04642,0.0519558,0.140884,0.10358,7.00062
7,0.065,0.175,0.0181777,0.269738,0.063542,0.0468223,0.130262,0.111458,7.00187
8,0.06,0.2,0.0178905,0.278134,0.0819878,0.0421095,0.121866,0.118012,7.00326
9,0.055,0.225,0.0172712,0.28494,0.101394,0.0377288,0.11506,0.123606,7.00477


We can now construct an interactive plot from this data:

In [52]:
from reaktplot import *

fig = Figure()
fig.title("AMOUNTS OF EXCHANGE SPECIES KX, CaX<sub>2</sub>, NaX")
fig.xaxisTitle('Ca<sup>2+</sup> [molal]')
fig.yaxisTitle('Amount [mol]')

fig.drawLineWithMarkers(df["amount_Ca"], df["amount_KX"], name="KX")
fig.drawLineWithMarkers(df["amount_Ca"], df["amount_CaX2"], name="CaX<sub>2</sub>")
fig.drawLineWithMarkers(df["amount_Ca"], df["amount_NaX"], name="NaX")

fig.show()

This figure shows that as the concentration of Ca<sup>2+</sup> increases from 0.0 to 0.5 molal, while the concentration of K<sup>+</sup> decreases from 0.1 to 0.0 molal, there is a tendency for the cations Na<sup>+</sup> and K<sup>+</sup> to unbind from their respective NaX and KX exchange phases. In the process, the cations Ca<sup>2+</sup> tend to bind to the available ion exchange species X<sup>-</sup>, with the exchange phase CaX{{_2}} becoming predominant among the others.

The next figure shows how the concentrations of Na<sup>+</sup>, K<sup>+</sup> and Ca<sup>2+</sup> behave in the process.

In [53]:
fig = Figure()
fig.title("CONCENTRATIONS OF IONS K<sup>+</sup>, Ca<sup>2+</sup>, Na<sup>+</sup>")
fig.xaxisTitle('Ca<sup>2+</sup> [molal]')
fig.yaxisTitle('Concentration [molal]')

fig.drawLineWithMarkers(df["amount_Ca"], df["amount_K_ion"], name="K<sup>+</sup>")
fig.drawLineWithMarkers(df["amount_Ca"], df["amount_Ca_ion"], name="Ca<sup>2+</sup>")
fig.drawLineWithMarkers(df["amount_Ca"], df["amount_Na_ion"], name="Na<sup>+</sup>")

fig.show()

Finally, we present how the pH of the solution changes in the process, whose changes are not significant.

In [54]:
fig = Figure()
fig.title("AQUEOUS SOLUTION pH")
fig.xaxisTitle('Ca<sup>2+</sup> [molal]')
fig.yaxisTitle('pH')

fig.drawLineWithMarkers(df["amount_Ca"], df["pH"], name="")

fig.show()